In [1]:
from minerl_rllib.envs.wrappers import wrap, gym_wrap

/home/shyam/anaconda3/envs/rayflow/lib/python3.7/site-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))


In [2]:
import gym
import ray

In [3]:
import numpy as np
import torch
import torch.optim as optim
import torch.nn as nn
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import torch.nn.functional as F

from ray import tune
from ray.tune.schedulers import ASHAScheduler

In [4]:
# Initialize Ray
ray.shutdown()

In [5]:
from rayflow import Rayflow
r = Rayflow.load("test_input.yml", minio_volume_path="./minio_volume/", mlflow_volume_path="./mlflow_volume/",no_ray=True)

HOST INFO {'192.168.1.65': {'ansible_python_interpreter': '~/anaconda3/envs/py37/bin/python', 'ansible_connection': 'local'}}


/home/shyam/anaconda3/envs/rayflow/lib/python3.7/site-packages/omegaconf/dictconfig.py:252: UserWarning: Keys with dot (127.0.0.1) are deprecated and will have different semantic meaning the next major version of OmegaConf (2.1)
See the compact keys issue for more details: https://github.com/omry/omegaconf/issues/152
You can disable this warning by setting the environment variable OC_DISABLE_DOT_ACCESS_WARNING=1
  warnings.warn(message=msg, category=UserWarning)


In [6]:
r.start()

/home/shyam/anaconda3/envs/rayflow/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
/home/shyam/anaconda3/envs/rayflow/lib/python3.7/site-packages/omegaconf/dictconfig.py:252: UserWarning: Keys with dot (192.168.1.65) are deprecated and will have different semantic meaning the next major version of OmegaConf (2.1)
See the compact keys issue for more details: https://github.com/omry/omegaconf/issues/152
You can disable this warning by setting the environment variable OC_DISABLE_DOT_ACCESS_WARNING=1
  warnings.warn(message=msg, category=UserWarning)


INPUT:  all:
  children:
    master_host:
      hosts:
        192.168.1.65:
          ansible_python_interpreter: ~/anaconda3/envs/py37/bin/python
          ansible_connection: local
    worker_hosts:
      hosts:
        192.168.1.65:
          ansible_python_interpreter: ~/anaconda3/envs/py37/bin/python
          ansible_connection: local
  vars:
    stdout_callback: debug
    display_args_to_stdout: true
    host_key_checking: false
    look_for_keys: true

PLAYBOOK Set up master hosts:
- name: Set up master hosts
  hosts: master_host
  gather_facts: true
  tasks:
  - name: startup-minio
    docker_container:
      name: rayflow_minio
      image: minio/minio:latest
      command: -c 'mkdir -p /minio_data/artifacts/ && /usr/bin/minio server --address
        :9001 /minio_data'
      ports:
      - 9001:9001
      detach: true
      tty: true
      interactive: true
      network_mode: host
      env:
        MINIO_ACCESS_KEY: test-access-key
        MINIO_SECRET_KEY: test-secret-ke

Using module file /home/shyam/anaconda3/envs/rayflow/lib/python3.7/site-packages/ansible/modules/cloud/docker/docker_container.py
<192.168.1.65> PUT /home/shyam/.ansible/tmp/ansible-local-103617_kyrbxig/tmpqj9srson TO /home/shyam/.ansible/tmp/ansible-tmp-1601412364.6587794-76013228363648/AnsiballZ_docker_container.py
<192.168.1.65> EXEC /bin/sh -c 'chmod u+x /home/shyam/.ansible/tmp/ansible-tmp-1601412364.6587794-76013228363648/ /home/shyam/.ansible/tmp/ansible-tmp-1601412364.6587794-76013228363648/AnsiballZ_docker_container.py && sleep 0'
<192.168.1.65> EXEC /bin/sh -c '~/anaconda3/envs/py37/bin/python /home/shyam/.ansible/tmp/ansible-tmp-1601412364.6587794-76013228363648/AnsiballZ_docker_container.py && sleep 0'
<192.168.1.65> EXEC /bin/sh -c 'rm -f -r /home/shyam/.ansible/tmp/ansible-tmp-1601412364.6587794-76013228363648/ > /dev/null 2>&1 && sleep 0'
changed: [192.168.1.65] => {
    "changed": true,
    "invocation": {
        "module_args": {
            "api_version": "auto",
    

INPUT:  all:
  children:
    master_host:
      hosts:
        192.168.1.65:
          ansible_python_interpreter: ~/anaconda3/envs/py37/bin/python
          ansible_connection: local
    worker_hosts:
      hosts: {}
  vars:
    stdout_callback: debug
    display_args_to_stdout: true
    host_key_checking: false
    look_for_keys: true

PLAYBOOK Set up master hosts:
- name: Set up master hosts
  hosts: master_host
  gather_facts: true
  tasks:
  - name: startup-minio
    docker_container:
      name: rayflow_minio
      image: minio/minio:latest
      command: -c 'mkdir -p /minio_data/artifacts/ && /usr/bin/minio server --address
        :9001 /minio_data'
      ports:
      - 9001:9001
      detach: true
      tty: true
      interactive: true
      network_mode: host
      env:
        MINIO_ACCESS_KEY: test-access-key
        MINIO_SECRET_KEY: test-secret-key
        MLFLOW_URL: http://192.168.1.65:5000
        MLFLOW_S3_ENDPOINT_URL: http://192.168.1.65:9001
        AWS_ACCESS_KEY

Using module file /home/shyam/anaconda3/envs/rayflow/lib/python3.7/site-packages/ansible/modules/cloud/docker/docker_container.py
<192.168.1.65> PUT /home/shyam/.ansible/tmp/ansible-local-103895kcv6h9fy/tmpnag9daak TO /home/shyam/.ansible/tmp/ansible-tmp-1601412380.0542479-152241885400204/AnsiballZ_docker_container.py
<192.168.1.65> EXEC /bin/sh -c 'chmod u+x /home/shyam/.ansible/tmp/ansible-tmp-1601412380.0542479-152241885400204/ /home/shyam/.ansible/tmp/ansible-tmp-1601412380.0542479-152241885400204/AnsiballZ_docker_container.py && sleep 0'
<192.168.1.65> EXEC /bin/sh -c '~/anaconda3/envs/py37/bin/python /home/shyam/.ansible/tmp/ansible-tmp-1601412380.0542479-152241885400204/AnsiballZ_docker_container.py && sleep 0'
<192.168.1.65> EXEC /bin/sh -c 'rm -f -r /home/shyam/.ansible/tmp/ansible-tmp-1601412380.0542479-152241885400204/ > /dev/null 2>&1 && sleep 0'
changed: [192.168.1.65] => {
    "ansible_facts": {
        "docker_container": {
            "AppArmorProfile": "docker-default"

<192.168.1.65> EXEC /bin/sh -c '~/anaconda3/envs/py37/bin/python /home/shyam/.ansible/tmp/ansible-tmp-1601412380.9328196-24798315409913/AnsiballZ_docker_container.py && sleep 0'
<192.168.1.65> EXEC /bin/sh -c 'rm -f -r /home/shyam/.ansible/tmp/ansible-tmp-1601412380.9328196-24798315409913/ > /dev/null 2>&1 && sleep 0'
changed: [192.168.1.65] => {
    "ansible_facts": {
        "docker_container": {
            "AppArmorProfile": "docker-default",
            "Args": [
                "server",
                "--backend-store-uri",
                "/mlflow",
                "--host",
                "192.168.1.65",
                "-p",
                "5000",
                "--default-artifact-root",
                "s3://artifacts/"
            ],
            "Config": {
                "AttachStderr": false,
                "AttachStdin": false,
                "AttachStdout": false,
                "Cmd": [
                    "mlflow",
                    "server",
              

In [7]:
ray.init(ignore_reinit_error=True,num_cpus=16, num_gpus=1)

2020-09-29 13:46:21,771	INFO resource_spec.py:231 -- Starting Ray with 33.45 GiB memory available for workers and up to 16.73 GiB for objects. You can adjust these settings with ray.init(memory=<bytes>, object_store_memory=<bytes>).
2020-09-29 13:46:22,196	INFO services.py:1193 -- View the Ray dashboard at localhost:8265


{'node_ip_address': '192.168.1.65',
 'raylet_ip_address': '192.168.1.65',
 'redis_address': '192.168.1.65:6379',
 'object_store_address': '/tmp/ray/session_2020-09-29_13-46-21_770287_103571/sockets/plasma_store',
 'raylet_socket_name': '/tmp/ray/session_2020-09-29_13-46-21_770287_103571/sockets/raylet',
 'webui_url': 'localhost:8265',
 'session_dir': '/tmp/ray/session_2020-09-29_13-46-21_770287_103571'}

In [ ]:
# from minerl_rllib.envs import register
# register()

### registering env

In [8]:
import copy
import os

import minerl
from ray.tune.registry import register_env

from minerl_rllib.envs.env import MineRLRandomDebugEnv, LazyMineRLEnv
from minerl_rllib.envs.wrappers import wrap, gym_wrap
from gym.envs.registration import register


def _register():
    """
    Must be called to register the MineRL environments for RLlib
    """

    for env_spec in minerl.herobraine.envs.obfuscated_envs:
        env_kwargs = copy.deepcopy(dict(
            observation_space=env_spec.observation_space,
            action_space=env_spec.action_space,
            docstr=env_spec.get_docstring(),
            xml=os.path.join(minerl.herobraine.env_spec.MISSIONS_DIR, env_spec.xml),
            env_spec=env_spec,
        ))

        def env_creator(env_config):
            return wrap(LazyMineRLEnv(**env_kwargs), **env_config)

        register_env(env_spec.name, env_creator)
        register(id="Custom"+env_spec.name, entry_point=gym_wrap(env_class=LazyMineRLEnv, env_kwargs=env_kwargs))
        
    def env_creator(env_config):
        return wrap(MineRLRandomDebugEnv(), **env_config)

    register_env('MineRLRandomDebug-v0', env_creator)

In [9]:
_register()

In [10]:
from ray.rllib.agents.ppo import PPOTrainer

train_name = "mlflow_train"
mlflow_logging_config = {
    "run_name":"rayflow_intial_test_run", 
    "run_tags":{},
    "experiment_name":"1_WORKER_TEST_PPO",
    "create_new_experiment": True
}
trainer_config = {
         "env": "MineRLNavigateDenseVectorObf-v0",
         "num_gpus":1,
         "num_workers": 1,
         "num_envs_per_worker": 1,
         "framework": "torch",
         'monitor':True, 
}
saver_config = {
        "checkpoint_on_end": True,
        "checkpoint_freq": 2
}
stop = {
        "training_iteration": 250,
}
tune_args = {
        "checkpoint_at_end": True,
        "checkpoint_freq": 25,
}

config = {
    "trainer_config":trainer_config,
    "logging_config": mlflow_logging_config
}

Instructions for updating:
non-resource variables are not supported in the long term


In [11]:
PPOTrainer.default_resource_request(trainer_config).to_json()

/home/shyam/anaconda3/envs/rayflow/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


{'cpu': 1,
 'gpu': 1,
 'memory': 0,
 'object_store_memory': 0,
 'extra_cpu': 1,
 'extra_gpu': 0,
 'extra_memory': 0,
 'extra_object_store_memory': 0,
 'custom_resources': {},
 'extra_custom_resources': {}}

In [12]:
from rayflow.tune import Tune
t = Tune()

In [ ]:
run_id, output_config, experiment_analysis = t.run_training(
               trainer=PPOTrainer, 
               name=train_name, 
               stop=stop, 
               mlflow_logging_config=mlflow_logging_config, 
               trainer_config=trainer_config,
               saver_config=saver_config,
               tune_args=tune_args,
               )

In [13]:
mlflow_logging_config = {
    "run_name":"rayflow_intial_test_run", 
    "run_tags":{},
    "experiment_name":"1_WORKER_TEST_PPO",
    "create_new_experiment": False
}

In [14]:
config = {
    "trainer_config":  trainer_config,
    "logging_config": mlflow_logging_config,
    "trainer": PPOTrainer,
}

In [17]:
mlflow_trainer = t.restore_trainer(PPOTrainer, trainer_config, mlflow_logging_config, run_id="c3266d73f0cf4781aea58726bd2cf25d", checkpoint_path="checkpoint_250", artifact_dir="./mlflow_artifacts/")

/home/shyam/anaconda3/envs/rayflow/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


ERROR! Session/line number was not unique in database. History logging moved to new session 119


Exception ignored in: <function _ConnectionBase.__del__ at 0x7fe4b159f1e0>
Traceback (most recent call last):
  File "/home/shyam/anaconda3/envs/rayflow/lib/python3.7/multiprocessing/connection.py", line 130, in __del__
    def __del__(self):
KeyboardInterrupt: 


KeyboardInterrupt: 

In [16]:
ppo_trainer = mlflow_trainer.trainer

/home/shyam/anaconda3/envs/rayflow/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [16]:
import gym
env = gym.make("CustomMineRLNavigateDenseVectorObf-v0")
episode_reward = 0
done = False
obs = env.reset()
for i in range(5000):
    action = ppo_trainer.compute_action(obs)
    obs, reward, done, info = env.step(action)
    episode_reward += reward
env.close()

/home/shyam/anaconda3/envs/rayflow/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
(pid=104326) /home/shyam/anaconda3/envs/rayflow/lib/python3.7/site-packages/gym/logger.py:30: UserWarning: WARN: Trying to monitor an environment which has no 'spec' set. This usually means you did not create it via 'gym.make', and is recommended only for advanced users.
(pid=104326)   warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))
(pid=104326) /home/shyam/anaconda3/envs/rayflow/lib/python3.7/site-packages/ray/rllib/utils/torch_ops.py:149: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly 

Traceback (most recent call last):
  File "/home/shyam/anaconda3/envs/rayflow/lib/python3.7/site-packages/IPython/core/interactiveshell.py", line 3417, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-17-febd47ea7b1d>", line 2, in <module>
    env = gym.make("CustomMineRLNavigateDenseVectorObf-v0")
  File "/home/shyam/anaconda3/envs/rayflow/lib/python3.7/site-packages/gym/envs/registration.py", line 145, in make
    return registry.make(id, **kwargs)
  File "/home/shyam/anaconda3/envs/rayflow/lib/python3.7/site-packages/gym/envs/registration.py", line 90, in make
    env = spec.make(**kwargs)
  File "/home/shyam/anaconda3/envs/rayflow/lib/python3.7/site-packages/gym/envs/registration.py", line 57, in make
    env = self.entry_point(**_kwargs)
  File "/home/shyam/anaconda3/envs/rayflow/lib/python3.7/site-packages/minerl_rllib-1.0.0-py3.7.egg/minerl_rllib/envs/wrappers/__init__.py", line 31, in wrap
    env = env_class(**env_kwargs)
  File "/home/s

TypeError: object of type 'NoneType' has no len()

In [ ]:
r.stop()

### Register MineRL Gym Environment to RLlib

In [ ]:
from minerl_rllib.envs import register

In [ ]:
register()

In [ ]:
from ray import tune
# https://docs.ray.io/en/latest/rllib-training.html#tuned-examples

import ray.rllib.agents.ppo as ppo
from ray.rllib.agents.ppo import PPOTrainer
# https://docs.ray.io/en/latest/rllib-training.html#tuned-examples
# config = ppo.DEFAULT_CONFIG.copy()
# config["num_gpus"] = 1
# config["num_workers"] = 1
# config["num_envs_per_worker"] = 1
# config["eager"] = False
# trainer = ppo.PPOTrainer(config=config, env="MineRLNavigateDenseVectorObf-v0")


tune.run(PPOTrainer,
         config={"env": "MineRLNavigateDenseVectorObf-v0",
                 "num_gpus": 1,
                 "num_workers": 1,
                 "num_envs_per_worker": 1,
                 "eager": False,
                 "use_pytorch": True,
#                  'monitor':True, 
#                  "model": {
#                     "custom_model": "fc_pov",
#                     }
                 }
         ,stop={"training_iteration": 3})  

In [ ]:
r.stop()